In [ ]:
# Connect Client
import tdmclient.notebook
await tdmclient.notebook.start()

# Import
import os
import sys
import numpy as np
import math

sys.path.insert(0, os.path.join(os.getcwd(), 'LocalNav'))
from LocalNav import LocalNav

# Init Position
#math.pi/2
abs_pos = [[20, 20, 0]] # absolute x,y,theta coordinates of the robot

# Goal Position
goal_position = [[80, 80]]

# Create Global Map
map_global = create_map_global()

# Main loop
while abs(np.sqrt( (abs_pos[0][0]-goal_position[0][0])*(abs_pos[0][0]-goal_position[0][0]) + (abs_pos[0][1]-goal_position[0][1])*(abs_pos[0][1]-goal_position[0][1]) ) ) > epsilon:
    
    # 2 States - Global / Local
    #if any(prox_horizontal):
    if any(value != float('inf') for value in prox_horizontal):
        state = 1 # Local
    else:
        state = 0 # Global
        

    if state == 1:
        closest_angle, map_global = localNav(abs_pos, goal_position, prox_horizontal, map_global)
        

